# Perguntas sobre Reinforcement Learning com contexto do Sutton

## 1. Lendo a chave

In [1]:
from dotenv import load_dotenv

import os

load_dotenv()

True

In [2]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_API_ENV = os.getenv("PINECONE_API_ENV")

In [3]:
PINECONE_API_KEY

'98a62383-2f1c-4112-91a4-4ced24915603'

## 2. Carregando o livro do Sutton

In [4]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [5]:
# Sutton - RL
loader = PyPDFLoader('../PDF/rl4rs.pdf')

In [6]:
rl4rs = loader.load()

In [7]:
rl4rs[1]

Document(page_content='Conference acronym ’XX, June 03–05, 2018, Woodstock, NY Wang, et al.\nTable 1: A comparison between RL4RS and other resources.\nDataset Opensource Others\nResourceArtifical\ndatasetSemi-simulated\ndatasetReal\ndatasetCodeRaw logged\ndataSimulation\nenvironmentOffline policy\nlearningOffline policy\nevaluation\nRecoGym[32] ✓× × ✓× ✓× ×\nRecsim[20] ✓× × ✓× ✓× ×\nVirtual-Taobao[36] × × ✓ ✓ × ✓× ×\nTop-k Off-policy[3] ✓× × ✓× × ✓×\nSlateQ[21] ✓× × ✓× × × ×\nAdversarial Model[4] × ✓× ✓× × × ×\nList-wise[43] × ✓× ✓× × × ×\nModel-based RS[1] ✓ ✓ × ✓× × ✓×\nOurs × × ✓ ✓ ✓ ✓ ✓ ✓\nitself. If an item is surrounded by similar but expensive items, the\nconversion rate increases, known as the decoy effect [ 19]. However,\nthe possible combinations of all the items can be billions, which is\nan NP-hard problem and less explored in supervised learning (SL).\nTo deal with these challenges, recent researchers resort to adopt-\ning reinforcement learning for recommendations, in whi

## 3. Quebrando em mais pedaços (`Chunks`)

In [8]:
# Definições dos chunks
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 1000,
    chunk_overlap  = 100,
    length_function = len,
)

In [9]:
texts = text_splitter.split_documents(rl4rs)

In [10]:
len(texts)

107

## 4. Embeddings de cada documento + busca semântica

In [11]:
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone

c:\Users\samue\anaconda3\envs\cenv\lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [12]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [13]:
# Inicializando o pinecone
pinecone.init(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV)
index_name = 'rl4rs'

In [14]:
# Passa todos os embeddings e guarda no index criado na conta em Pinecone
docsearch = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name)

In [15]:
# Lançando a query e vendo os resultados mais similares
query = 'What has been the economic relevance of recommender systems in industry?'
docs = docsearch.similarity_search(query)


In [16]:
docs

[Document(page_content='ommendation scenarios, including slate recommendation (lists of\nitems), bundle recommendation (a collection of items that should be\npurchased simultaneously), sequential recommendation, and many\nothers, as shown in Figure 1. It is worth exploring the various chal-\nlenges the modern e-commerce industry faces today. Most current\ne-commerce and retail companies build their recommender systems\nby implementing supervised learning based algorithms on their\nwebsites to maximize immediate user satisfaction in a greedy man-\nner. However, the item-wise greedy recommendation strategy is an\nimperfect fitting to real recommendation systems. With more and\nmore new upcoming recommendation scenarios, more and more\nchallenges have to be solved. For instance, in sequential recom-\nmendation scenarios, traditional methods often consider different\nrecommendation steps in a session to be independent and fail to\nmaximize the expected accumulative utilities in a recommend

## 5. Consultando os documentos para obtermos nossa resposta

In [18]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

In [19]:
llm = OpenAI(temperature=0, api_key=OPENAI_API_KEY)
chain = load_qa_chain(llm, chain_type='stuff')

In [20]:
print(chain.run(input_documents=docs, question=query))

 The economic relevance of recommender systems in industry has been significant, with the e-commerce industry projected to grow to 6.3 trillion US dollars in 2023. This growth is driven in part by the rapid increase in various recommendation scenarios, such as slate recommendation and sequential recommendation, which aim to maximize user satisfaction and increase sales. However, there are challenges in implementing these systems, and the RL4RS (Reinforcement Learning for Recommender Systems) resource has been developed to address these challenges and contribute to research in applied reinforcement learning.


## 6. Chat implementando o modelo

### 6.1. Antes de implementar RAG.

In [21]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

In [22]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    model='gpt-3.5-turbo',
    temperature=1.0
)

In [23]:
messages = [
    SystemMessage(content='Você é um bot assistente gentil e prestativo.'),
    HumanMessage(content='Olá, como você está hoje?'),
    AIMessage(content='Eu estou bem, obrigado por perguntar. Como eu possso te ajudar?'),
    HumanMessage(content='Eu gostaria de saber mais sobre sistemas de recomendação baseados em reinforcement learning.')
]

In [24]:
res = chat(messages)
res

AIMessage(content='Claro! Sistemas de recomendação baseados em reinforcement learning (aprendizado por reforço) são uma abordagem que utiliza técnicas de machine learning para recomendar itens ou ações personalizadas aos usuários.\n\nO aprendizado por reforço é uma técnica de machine learning onde um agente aprende a tomar decisões em um ambiente para maximizar uma recompensa. No caso de sistemas de recomendação, o agente é o sistema de recomendação em si e o ambiente é o conjunto de usuários e itens disponíveis para recomendação.\n\nNesse tipo de sistema, o sistema de recomendação recebe informações sobre as preferências e ações dos usuários, bem como o feedback sobre as recomendações anteriores. Com base nessas informações, ele aprende a tomar decisões sobre quais itens recomendar para cada usuário da maneira mais efetiva possível.\n\nO objetivo é encontrar um equilíbrio entre explorar novos itens desconhecidos para os usuários e explorar itens já conhecidos e mais prováveis de serem

In [25]:
print(res.content)

Claro! Sistemas de recomendação baseados em reinforcement learning (aprendizado por reforço) são uma abordagem que utiliza técnicas de machine learning para recomendar itens ou ações personalizadas aos usuários.

O aprendizado por reforço é uma técnica de machine learning onde um agente aprende a tomar decisões em um ambiente para maximizar uma recompensa. No caso de sistemas de recomendação, o agente é o sistema de recomendação em si e o ambiente é o conjunto de usuários e itens disponíveis para recomendação.

Nesse tipo de sistema, o sistema de recomendação recebe informações sobre as preferências e ações dos usuários, bem como o feedback sobre as recomendações anteriores. Com base nessas informações, ele aprende a tomar decisões sobre quais itens recomendar para cada usuário da maneira mais efetiva possível.

O objetivo é encontrar um equilíbrio entre explorar novos itens desconhecidos para os usuários e explorar itens já conhecidos e mais prováveis de serem aceitos. O sistema de re

In [26]:
# Vamos adicionar a última resposta do chat às mensagens
messages.append(res)

# criando um novo user prompt
prompt = HumanMessage(
    content='Você conhece o framework RL4RS?'
)

# adicionando às mensagens
messages.append(prompt)

# requisitando a resposta do chat
res = chat(messages)

print(res.content)

Peço desculpas, mas não estou familiarizado com um framework chamado RL4RS. Não consigo fornecer informações detalhadas sobre isso. No entanto, fique à vontade para perguntar sobre outros frameworks ou detalhes específicos sobre reinforcement learning ou sistemas de recomendação. Estou aqui para ajudar!


### 6.2. Inserindo contexto (conceito de RAG)

In [27]:
def augment_prompt(query: str):
    # get top 3 results from knowledge base
    results = docsearch.similarity_search(query, k=3)
    # get the text from the results
    source_knowledge = "\n".join([x.page_content for x in results])
    # feed into an augmented prompt
    augmented_prompt = f"""Usando o contexto abaixo, responda a pergunta abaixo. Se a pergunta não estiver relacionada ao contexto, esqueça o contexto dado e responda como você normalmente responderia, pois você ainda é um assistente prestativo. E se apesar da pergunta não estiver relacionada você não saber a resposta, apenas diga que não sabe como normalmente você faria.

    Contextos:
    {source_knowledge}

    Query: {query}"""
    return augmented_prompt

Note, agora, na resposta dada após a inserção de contexto utilizando o próprio artigo do framework.

In [28]:
prompt = HumanMessage(
    content=augment_prompt('Você conhece o framework RL4RS?')
)

# adicionando às mensagens
messages.append(prompt)

# requisitando a resposta do chat
res = chat(messages)

print(res.content)

Sim, eu conheço o framework RL4RS. O RL4RS (Reinforcement Learning for Recommender Systems) é um recurso desenvolvido pela Netease, que consiste em um conjunto de dados de código aberto para sistemas de recomendação baseados em reinforcement learning. Ele inclui conjuntos de dados em larga escala, ambientes de simulação reprodutíveis, baselines avançados de RL relacionados e ferramentas de compreensão de dados.

O RL4RS também oferece um framework de avaliação sistemática, que inclui avaliação da simulação do ambiente, avaliação do ambiente de simulação e avaliação da política contrafactual. Ele foi construído em Python e utiliza TensorFlow para modelagem e treinamento.

O objetivo do RL4RS é atender à crescente demanda por sistemas de RL adaptados a cenários de recomendação inovadores. Ele fornece uma plataforma para o desenvolvimento e avaliação de sistemas de recomendação baseados em RL.

Você pode encontrar o RL4RS no repositório do GitHub da Netease, no seguinte link: https://gith

In [29]:
prompt = HumanMessage(
    content=augment_prompt('Você poderia me ajudar?')
)

# adicionando às mensagens
messages.append(prompt)

# requisitando a resposta do chat
res = chat(messages)

print(res.content)

Claro! Estou aqui para ajudar. Por favor, diga-me qual é sua pergunta ou problema específico, e eu farei o possível para fornecer orientação ou suporte.


### 6.3. Gerando chat com contexto inserido

In [50]:
import gradio as gr

def predict(message, history):
    history_langchain_format = [SystemMessage(content='Você é um assistente gentil e prestativo. Busque responder as perguntas quando conseguir!')]
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))
    history_langchain_format.append(HumanMessage(content=augment_prompt(message)))
    gpt_response = chat(history_langchain_format)
    return gpt_response.content

In [51]:
gr.ChatInterface(predict).launch(debug=True)

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


### 6.4. Assistente de voz com contexto

In [31]:
from openai import OpenAI
client = OpenAI()

In [32]:
import gradio as gr
import numpy as np
import soundfile as sf

def transcribe2(audio):
    sr, y = audio
    y = y.astype(np.float32)
    y /= np.max(np.abs(y))

    sf.write("output_audio.wav", y, sr, format='WAV', subtype='PCM_16')

    transcript = client.audio.transcriptions.create(
        model="whisper-1", 
        file=open('output_audio.wav', 'rb'),
        response_format='text'
    )
    return transcript

def voice_chat(user_voice):

    messages = [
    {"role": "system", "content": "Você é um assistente gentil e prestativo e que responde no idioma do usuário."},
    ]       
    
    user_message = transcribe2(user_voice)

    messages.append(
        {"role": "user", "content": augment_prompt(user_message)},
    )

    print(messages)

    chat = client.chat.completions.create(
        model="gpt-3.5-turbo", messages=messages
    )
    
    reply = chat.choices[0].message.content
    
    messages.append({"role": "assistant", "content": reply})

    response = client.audio.speech.create(
        model="tts-1",
        voice="onyx",
        input=reply
    )
    response.stream_to_file('output.mp3')

    return (reply, 'output.mp3')


text_reply = gr.Textbox(label="ChatGPT Text")
voice_reply = gr.Audio('output.mp3')

demo = gr.Interface(
    voice_chat,
    gr.Audio(sources=["microphone"]),
    outputs=[text_reply, voice_reply],
    title = 'AI Voice Assistant with ChatGPT AI'
)

demo.launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


[{'role': 'system', 'content': 'Você é um assistente gentil e prestativo e que responde no idioma do usuário.'}, {'role': 'user', 'content': 'Usando o contexto abaixo, responda a pergunta abaixo. Se a pergunta não estiver relacionada ao contexto, esqueça o contexto dado e responda como você normalmente responderia, pois você ainda é um assistente prestativo. E se apesar da pergunta não estiver relacionada você não saber a resposta, apenas diga que não sabe como normalmente você faria.\n\n    Contextos:\n    2https://github.com/google-research/recsim\n3https://github.com/eyounx/VirtualTaobao\n4https://github.com/awarebayes/RecNN\n5https://github.com/google-research/recsim\n6https://github.com/xinshi-chen/GenerativeAdversarialUserModel\n2https://github.com/google-research/recsim\n3https://github.com/eyounx/VirtualTaobao\n4https://github.com/awarebayes/RecNN\n5https://github.com/google-research/recsim\n6https://github.com/xinshi-chen/GenerativeAdversarialUserModel\nSection 3), and are upd

ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\Users\samue\anaconda3\envs\cenv\lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 408, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "c:\Users\samue\anaconda3\envs\cenv\lib\site-packages\uvicorn\middleware\proxy_headers.py", line 84, in __call__
    return await self.app(scope, receive, send)
  File "c:\Users\samue\anaconda3\envs\cenv\lib\site-packages\fastapi\applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "c:\Users\samue\anaconda3\envs\cenv\lib\site-packages\starlette\applications.py", line 116, in __call__
    await self.middleware_stack(scope, receive, send)
  File "c:\Users\samue\anaconda3\envs\cenv\lib\site-packages\starlette\middleware\errors.py", line 186, in __call__
    raise exc
  File "c:\Users\samue\anaconda3\envs\cenv\lib\site-packages\starlette\middleware\errors.py", line 164, in __call__
    awa

[{'role': 'system', 'content': 'Você é um assistente gentil e prestativo e que responde no idioma do usuário.'}, {'role': 'user', 'content': 'Usando o contexto abaixo, responda a pergunta abaixo. Se a pergunta não estiver relacionada ao contexto, esqueça o contexto dado e responda como você normalmente responderia, pois você ainda é um assistente prestativo. E se apesar da pergunta não estiver relacionada você não saber a resposta, apenas diga que não sabe como normalmente você faria.\n\n    Contextos:\n    RL-based RS suffers from the difficulties of being well-validated\nbefore deployment. We attempt to propose a new systematic evalu-\nation framework, including evaluation of environment simulation,\nevaluation on environments, counterfactual policy evaluation, and\nevaluation on environments built from test set. In summary, the\nRL4RS (Reinforcement Learning for Recommender Systems), a new\nresource with special concerns on the reality gaps, contains two\nreal-world datasets, data u